
___________
### 0. Delete database

In [10]:
import os
import sqlite3

# Close any existing connections
conn = sqlite3.connect('nursery.db')
conn.close()

# Delete the file completely
if os.path.exists('nursery.db'):
    os.remove('nursery.db')
    print("Database file removed completely!")
else:
    print ("Database file did not exist.")

Database file removed completely!


In [ ]:
import sqlite3

# Connect to your database
conn = sqlite3.connect('nursery.db')
cursor = conn.cursor()

# To DROP specific tables
try:
    cursor.execute('''
        DROP TABLE IF EXISTS SPECIES
    ''')
    conn.commit()  # Important: Save the changes
    print("Table dropped successfully")
    
except sqlite3.Error as e:
    print(f"Error dropping table: {e}")
    
finally:
    conn.close()  # Always close the connection

Table dropped successfully


______________________
### 1. Database creation and table architecture

In [11]:
import sqlite3

# Connect to database
conn = sqlite3.connect('nursery.db')
cursor = conn.cursor()

try:
    # 1. SPECIES table for static, descriptive attributes of a species
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS SPECIES (
            ID INTEGER PRIMARY KEY AUTOINCREMENT,
            SCIENTIFIC_NAME TEXT NOT NULL UNIQUE, -- The botanical name
            COMMON_NAME TEXT, -- The common name
            FAMILY TEXT, 
            GERMINATION_DAYS_AVG INTEGER,
            GROWTH_RATE TEXT CHECK(GROWTH_RATE IN ('Slow', 'Moderate', 'Fast')),
            WATER_NEEDS TEXT CHECK(WATER_NEEDS IN ('Low', 'Medium', 'High')),
            LIGHT_REQUIREMENTS TEXT CHECK(LIGHT_REQUIREMENTS IN ('Full Sun', 'Partial Shade', 'Shade')),
            NOTES TEXT
        )            
    ''')

    # 2. BATCH Table for unmutable and distinct batch count 
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS BATCH (
            ID INTEGER PRIMARY KEY AUTOINCREMENT,
            SPECIES_ID INTEGER NOT NULL,
            BATCH_CODE TEXT NOT NULL UNIQUE,
            START_DATE DATE NOT NULL,
            MOON_PHASE TEXT CHECK (MOON_PHASE IN ('New', 'First Quarter', 'Full', 'Last Quarter')),
            INITIAL_SEED_COUNT INTEGER NOT NULL,
            SPROUT_DATE DATE,
            BAGGING_DATE DATE,
            INITIAL_PLANT_COUNT INTEGER,
            NOTES TEXT,
            FOREIGN KEY (SPECIES_ID) REFERENCES SPECIES(ID)
        )
    ''')

    # 3. BATCH_OBSERVATION table Focused on plant counts & health
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS BATCH_OBSERVATION(
            ID INTEGER PRIMARY KEY AUTOINCREMENT,
            BATCH_ID INTEGER NOT NULL,
            OBSERVATION_DATE DATE NOT NULL,
            MOON_PHASE TEXT CHECK (MOON_PHASE IN ('New', 'First Quarter', 'Full', 'Last Quarter')),
            LIVE_PLANT_COUNT INTEGER NOT NULL,
            HEIGHT_AVG_CM REAL,
            HEALTH_INDICATOR TEXT CHECK(HEALTH_INDICATOR IN ('Excellent', 'Good', 'Fair', 'Poor')),
            NOTES TEXT,
            FOREIGN KEY (BATCH_ID) REFERENCES BATCH(ID),
            UNIQUE(BATCH_ID, OBSERVATION_DATE)  
        )
    ''')

    # 4. WATERING_LOG table focused in irrigation
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS WATERING_LOG (
            ID INTEGER PRIMARY KEY AUTOINCREMENT,
            BATCH_ID INTEGER NOT NULL,
            WATERING_TIMESTAMP DATETIME NOT NULL,
            WATER_AMOUNT_LT REAL,
            METHOD TEXT CHECK(METHOD IN('Sprinkler', 'Drip', 'Bottle Watering', 'Hose' )),
            NOTES TEXT,
            FOREIGN KEY (BATCH_ID) REFERENCES BATCH(ID)
        )
    ''')

    # 5. WEATHER_LOG table to track separatedly rain and other relevant weather conditions
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS WEATHER_LOG (
            ID INTEGER PRIMARY KEY AUTOINCREMENT,
            TIMESTAMP_START DATETIME NOT NULL,
            TIMESTAMP_END DATETIME NOT NULL,
            PRECIPITATION_CATEGORY TEXT CHECK (PRECIPITATION_CATEGORY IN ('None', 'Heavy Rain', 'Moderate Rain', 'Persistent Drizzle')),
            MAX_TEMP_C REAL,
            MIN_TEMP_C REAL,
            NOTES TEXT
        )
    ''')

    # 6 PEST_MONITORING table for pest control and efficiency over time
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS PEST_MONITORING_LOG (
            ID INTEGER PRIMARY KEY AUTOINCREMENT,
            BATCH_ID INTEGER NOT NULL,
            PEST_NAME TEXT,
            SEVERITY_LEVEL TEXT CHECK(SEVERITY_LEVEL IN ('None', 'Low', 'Medium', 'High')),
            APPLICATION_TIMESTAMP DATETIME NOT NULL,
            MOON_PHASE TEXT CHECK (MOON_PHASE IN ('New', 'First Quarter', 'Full', 'Last Quarter')),
            PRODUCT_NAME TEXT,   
            APPLICATION_METHOD TEXT,
            PRODUCT_DOSE REAL
        )
    ''')

    # 7 COSTS table to track the operational expenses
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS EXPENSES (
            ID INTEGER PRIMARY KEY AUTOINCREMENT,
            DATE DATE NOT NULL,
            CATEGORY TEXT NOT NULL,
            DESCRIPTION TEXT NOT NULL,
            AMOUNT REAL NOT NULL,
            NOTES TEXT
        )
    ''')


    conn.commit()
    print("Database schema created successfully!")

except sqlite3.Error as e:
    print(f"An error occurred : {e}")
    conn.rollback()

finally:
    conn.close()

Database schema created successfully!
